# Fix Records Length
将不定长的记录文件转换成定长记录文件，每行字段为：`path, x0, y0, x1, y1, label, score`
  
Origin records: `path0, (x000, y000, x001, y001, label00, score00), (x010, y010, x011, y011, label01, score01) ... \n path1, (x100, y100, x101, y101, label10, score10)...`
  
Formated records: `path0, x000, y000, x001, y001, label00, score00 \n path0, x010, y010, x011, y011, label01, score01 \n ...`

用法：
1. 将变量`ORG_RECORD`的值修改为原始不定长记录文件；
2. 将变量`SAVE_TO`的值修改为转换成定长记录后要保存的地址；
3. 将需要对每行不定长记录做的预处理操作写在`prework`函数中；
4. 最后直接运行所有程序即可。

In [17]:
ORG_RECORD = "./jilu.txt"
SAVE_TO = "./label.txt"


def prework(record_line):
    # change absolute path into relative path
    record_line = record_line.replace('/nfs/syzhou/github/yolo3_hand/', './')
    return record_line

"""Split Line
=====================================================================================
"""

import os
import numpy as np
from warnings import warn


def fix_length():
    metadata = []
    if os.path.isfile(ORG_RECORD):
        with open(ORG_RECORD) as fp:
            for i, itm in enumerate(fp.readlines()):
                itm = prework(itm)
                inpt = itm.split()
                if len(inpt) % 6 != 1:
                    warn('[!]Warning: format error in line %d (%s)' % (i, itm))
                try:
                    path = inpt[0]
                    bboxes = np.array([float(d)
                                       for d in inpt[1:]]).reshape(-1, 6)
                except Exception as e:
                    warn("Error in line %d (%s): %s!" % (i, itm, str(e)))
                for bbx in bboxes:
                    x0, y0, x1, y1, label, score = bbx.tolist()
                    if bbx[-2] > 3:
                        x1, y1, label = label, x1, y1
                    metadata.append([
                        path,
                        int(x0),
                        int(y0),
                        int(x1),
                        int(y1),
                        int(label), score
                    ])
        try:
            with open(SAVE_TO, 'w+') as fo:
                for itm in metadata:
                    for elm in itm:
                        fo.write(str(elm) + ' ')
                    fo.write('\n')
            print("Saved into " + SAVE_TO)
        except Exception as e:
            warn("Error in save data into " + SAVE_TO)
    else:
        metadata = []
        warn("Error: %s is not available file!" % ORG_RECORD)


fix_length()

Saved into ./label.txt
